# Analyzing a transmon qubit and a resonator
### Prerequisite
A working local installation of Ansys

### Transmon & resonator: render into Ansys.
1. Run finite element eigenmode analysis. <br>
1. Plot fields and display them. <br>
1. Set up EPR junction dictionary. <br>
1. Run EPR analysis on the two modes. <br>
1. Get qubit frequency and anharmonicity. <br>
1. Render to Ansys Q3D. <br>
1. Calculate the capacitance matrix. <br>

In [ ]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

In [ ]:
%metal_heading Create a single transmon qubit and a resonator

### Create the design in Metal
Create a design by specifying the chip size and open Metal GUI.

In [ ]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
hfss = design.renderers.hfss

gui = MetalGUI(design)

Create a single transmon with one readout resonator. Please refer to tutorials on analyzing a transmon and analyzing a resonator if you're not familiar with the cell below.

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

design.delete_all_components()

q1 = TransmonPocket(design, 'Q1', options = dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=+1, pad_width='200um')
    )))
otg = OpenToGround(design, 'open_to_ground', options=dict(pos_x='1.75mm',  pos_y='0um', orientation='0'))
readout = RouteMeander(design, 'readout',  Dict(
            total_length='6 mm',
            hfss_wire_bonds = True,
            fillet='90 um',
            lead = dict(start_straight='100um'),
            pin_inputs=Dict(
                start_pin=Dict(component='Q1', pin='readout'),
                end_pin=Dict(component='open_to_ground', pin='open')), ))

gui.rebuild()
gui.autoscale()

### Establish a connection between Metal and Ansys
Open Ansys. This can either be done manually, or by executing one of the lines in the following cell. <br>
open_ansys() will look by default for the 2021 R2 version of Ansys. You can easily reroute it to your Ansys of choice by providing the name of the environment variable that contains the path (path_env) or the path itself (path)

In [ ]:
hfss.open_ansys()   # this opens Ansys 2021 R2 if present
# hfss.open_ansys(path_var='ANSYSEM_ROOT211')
# hfss.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM21.1\Win64')
# hfss.open_ansys(path='../../../Program Files/AnsysEM/AnsysEM21.1/Win64')

If a project was not automatically opened, you can manually create/open one from the Ansys GUI. Alternatively you can execute the following cell.

In [ ]:
# hfss.new_ansys_project()

Finally connect this Jupyter Notebook session with the active Ansys project.

In [ ]:
hfss.connect_ansys()
# hfss.connect_ansys('C:\\project_path\\', 'Project1')  # will open a saved project before linking the Jupyter session

### Prepare and run the analysis

Create and activate an eigenmode design called "TransmonReadout".

In [ ]:
hfss.activate_eigenmode_design("TransmonReadout")

Render everything inlcuding the qubit and resonator in Metal, to "TransmonReadout" design in Ansys.

In [ ]:
hfss.render_design(['Q1', 'readout', 'open_to_ground'], [])
hfss.save_screenshot()

Set the convergence parameters and junction properties in the Ansys design. Then run the analysis and plot the convergence. Note that we seek 2 eigenmodes - one with stronger fields near the transmon, the other with stronger fields near the resonator.

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.n_modes = 2
setup.passes = 20
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

### Plot fields and display them
Display the Ansys modeler window and plot the E-field on the chip's surface.

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

Delete the newly created E-field plot before moving on.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

To look at the second eigenmode created, we use the following command, and then plot the corresponding E-field.

In [ ]:
hfss.set_mode(2, "Setup")

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

We delete this design to prepare for further analysis.

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

In [ ]:
%metal_print Energy-Participation-Ratio Analysis

### Set up EPR parameters
Specify the junctions in the model; in this case there's only one, namely 'jj'.

In [ ]:
#Non-linear (Josephson) junction
pinfo = hfss.pinfo # Project info
pinfo.junctions['jj'] = {'Lj_variable': 'Lj', 'rect': 'JJ_rect_Lj_Q1_rect_jj', 
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj'}
pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied
pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify
# Handles  microwave analysis on eigenmode solutions
eprd = epr.DistributedAnalysis(pinfo)

Find the electric and magnetic energy stored in the substrate and the system as a whole.

In [ ]:
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

### Run EPR analysis

Perform EPR analysis for all modes and variations.

In [ ]:
eprd.do_EPR_analysis()

# 4a. Perform Hamiltonian spectrum post-analysis, building on mw solutions using EPR
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 7)

# 4b. Report solved results
swp_variable = 'Lj' # suppose we swept an optimetric analysis vs. inductance Lj_alice
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

Release Ansys session

In [ ]:
hfss.disconnect_ansys()

In [ ]:
%metal_print Q3D Analysis

Next we will look at a lumped oscillator model (LOM) for the same design.

## Render to Ansys Q3D

In [ ]:
q3d = design.renderers.q3d

Again, we establish a connection with Ansys and add design for analysis

In [ ]:
q3d.connect_ansys()
q3d.activate_q3d_design("TransmonResonator_q3d")

Next, we render the exisitng design to Ansys Q3D for analysis. To ensure that the readout is insulated from the ground plane, we set the 'readout' pin of Q1 to have an open termination.

In [ ]:
q3d.render_design(['Q1'], [('Q1','readout')])

## Use Ansys Q3D to obtain the capacitance matrix

In [ ]:
#Analyze the default solution setup
q3d.analyze_setup("Setup")

For a transmon only case, we should see a 4x4 capacitance matrix, four conductors corresponding to two conducting pads, one connector pad, and the ground palne.

In [ ]:
# Using the analysis results, get its capacitance matrix as a dataframe.
q3d.get_capacitance_matrix()

The optional keyword parameters of get_capacitance_matrix are given below: <br><br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
pass_number: int = 3

In [ ]:
# Run lumped oscillator model (LOM) simulations and save results in dict_lom.b
dict_lom = q3d.lumped_oscillator_vs_passes(12.31, 2, 1, 7.0, [], 9)
dict_lom

Using capacitance matrices obtained from each pass, save the many parameters of the Hamiltonian of the system. lumped_oscillator_vs_passes takes in the following parameters: <br><br>
Lj_nH: float <br>
Cj_fF: float <br>
N: int <br>
fr: Union[list, float] <br>
fb: Union[list, float] <br>
maxPass: int <br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
g_scale: float = 1

Here,<br>
Lj - the Josephson inductance of your Josephson junction <br>
Cj - the capacitance of your Josephson junction<br>
N - the total number of connection pads <br>
fr - the frequency of the readout resonator <br>
[fb1, fb2, fb3...., fbN-1] - list of the frequencies of the busses<br>
passes - the number of passes your simulation ran for

dist_analysis allows one to calculate dissipation and other properties. It is used for the convergence plots below.

In [ ]:
q3d.plot_convergence_main(dict_lom);
q3d.plot_convergence_chi(dict_lom)

In [ ]:
q3d.disconnect_ansys()

In [ ]:
gui.main_window.close()